In [1]:
import pymongo
import pandas as pd
client = pymongo.MongoClient('da1.eecs.utk.edu')
data = list(client['fdac19-Stackbot']['SOdata'].find())
df = pd.DataFrame(data)
removed = []
data = pd.DataFrame() # Used in the Random Forest Model

# print(len(df)) # Length before cleaning

# Finds all dictionaries in df['code_and_ast'] that need to be removed
# i.e. all questions that do not contain any code
for i in df.index:
    for j in df['code_and_ast'][i]:
        if j['ast'] == 'none':
            removed.append((i, j))
            
# Handles the removal process from df using the removed list
for x in removed:        
    if (df.at[x[0], 'code_and_ast'] is not None):
        (df.at[x[0], 'code_and_ast']).remove(x[1])
        if len(df.at[x[0], 'code_and_ast']) == 0:
            df.at[x[0], 'code_and_ast'] = None
    else:
        print('Something went wrong! :(')

# Removes all df rows that are non-null
df = df[df['code_and_ast'].notnull()] 

df.reset_index(inplace=True) # Resets index from 1, 3, 5 --> 1, 2, 3 etc.

# Build a static list of tags (that need to be predicted) based off of the df
# Input feature set: 
for i in df.index:
    for cat in df.at[i, 'code_and_ast']:
        for tag in df.at[i, 'tags']:
            if cat['ast'] is not None:
                d = {'ast': [cat['ast']], 'tag': [tag]}
                # Wow nice PANDAS this is not how append should works
                data = data.append(pd.DataFrame(d), ignore_index=True)

In [ ]:
from tqdm import tqdm
from pd_doc2vec_threaded import doc2vec

# Converts the data tag field into a set for float usage within our model
tag_set = set(data['tag'])

tag_dict = {}
for i, x in enumerate(tag_set):
    tag_dict.update({x:i})

data['tag_val'] = None
for x in data.index:
    data.at[x, 'tag_val'] = tag_dict[data.at[x, 'tag']]
    
# Removes python popular tag
removed = []
for x in data.index:
    if 'python' in data.at[x, 'tag']:
        removed.append(x)
data.drop(removed, inplace=True)

# Generalizes tags i.e. pyqt5 falls under pyqt, etc.
for x in tqdm(data.index):
    for y in data.index:
        if data.at[x, 'tag'] in data.at[y, 'tag'] and x != y:
            data.at[y, 'tag'] = data.at[x, 'tag']
            data.at[y, 'tag_val'] = data.at[x, 'tag_val']
            
print(data)

# This is our model
# stack_brain = doc2vec(data, 'ast', ['tag_val'])

 38%|███▊      | 3471/9051 [09:51<15:34,  5.97it/s]

In [ ]:
# This is the training
stack_brain.score(verbose=True, iterations=10)